#### Под словом будем понимать слово до проведения его нормализации с помощью стемминга/лемматизации

In [3]:
import re

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, length, avg, lower, regexp_replace

In [5]:
# Создание SparkSession
spark = SparkSession.builder.getOrCreate()

# Загрузка данных из файла
file_path = "wiki.txt"
df = spark.read.option("delimiter", "\t").csv(file_path, header=False).toDF("url", "title", "text")

# Разбивка текста на слова
words_df = df.select(explode(split(col("text"), "\\s+")) \
                     .alias("word"))

In [6]:
# 1. Найти самое длинное слово
longest_word = words_df \
    .filter(col("word").rlike(r"^[a-zA-Zа-яА-Я]+$")) \
    .withColumn("length", length(col("word"))) \
    .orderBy(col("length").desc()) \
    .select("word") \
    .first()

print(f"Самое длинное слово: {longest_word['word']}")

Самое длинное слово: никотинамидадениндинуклеотида


In [7]:
# 2. Найти среднюю длину слов
average_word_length = words_df.withColumn("length", length(col("word"))) \
    .filter(col("word").rlike(r"^[a-zA-Zа-яА-Я]+$")) \
    .agg(avg(col("length")).alias("average_length")) \
    .first()

print(f"Средняя длина слов: {average_word_length['average_length']}")

Средняя длина слов: 6.198729955995877


In [25]:
cleaned_words_df = words_df.withColumn("word", regexp_replace(lower(col("word")), r"[^a-z]", ""))
latin_words_df = cleaned_words_df.filter(col("word").rlike("^[a-z]+$"))
word_count_df = latin_words_df.groupBy("word").count()
most_common_word = word_count_df.orderBy(col("count").desc()).first()

print(f"Самое частоупотребляемое латинское слово: {most_common_word['word']} ({most_common_word['count']} раз)")

Самое частоупотребляемое латинское слово: formula (11559 раз)


In [9]:
# 4. Слова, которые более чем в половине случаев начинаются с большой буквы и встречаются больше 10 раз
lower_words_df = words_df.withColumn("lower_word", lower(col("word")))

total_count_df = lower_words_df.groupBy("lower_word").count().alias("total_count")
capitalized_words_df = words_df.filter(col("word").rlike("^[A-ZА-Я].*"))
capitalized_lower_words_df = capitalized_words_df.withColumn("lower_word", lower(col("word")))
capitalized_count_df = capitalized_lower_words_df.groupBy("lower_word").count().alias("capitalized_count")

combined_df = total_count_df.join(capitalized_count_df, on="lower_word") \
    .withColumn("capitalized_fraction", col("capitalized_count.count") / col("total_count.count")) \
    .filter((col("capitalized_fraction") > 0.5) & (col("total_count.count") > 10))

result_df = combined_df.select("lower_word", "capitalized_fraction", "total_count.count")
result_df.show(truncate=False)


+----------+--------------------+-----+
|lower_word|capitalized_fraction|count|
+----------+--------------------+-----+
|ab        |0.5625              |16   |
|access    |0.9473684210526315  |19   |
|acid      |0.6666666666666666  |15   |
|acl       |1.0                 |13   |
|adsl      |1.0                 |24   |
|ag        |1.0                 |12   |
|ag,       |1.0                 |16   |
|airbus    |1.0                 |13   |
|airlines, |1.0                 |13   |
|alpha,    |1.0                 |12   |
|alt       |1.0                 |55   |
|am5x86    |1.0                 |11   |
|american  |1.0                 |47   |
|amigaos   |1.0                 |16   |
|android,  |1.0                 |11   |
|anime     |1.0                 |16   |
|apache    |0.9714285714285714  |35   |
|api       |1.0                 |73   |
|api.      |1.0                 |26   |
|apple     |1.0                 |123  |
+----------+--------------------+-----+
only showing top 20 rows



In [ ]:
# 5. Определить устойчивые сокращения вида пр., др.
abbreviations_df = words_df.filter(col("word").rlike("^[а-я]{1,3}\.$"))
abbreviations_df.groupBy("word") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(truncate=False)

+----+-----+
|word|count|
+----+-----+
|тыс.|4211 |
|г.  |2806 |
|лет.|1562 |
|др. |1276 |
|год.|1055 |
|гг. |961  |
|н.  |943  |
|им. |906  |
|век.|876  |
|чел.|759  |
|э.  |579  |
|т.  |439  |
|в.  |358  |
|руб.|351  |
|км. |350  |
|с.  |337  |
|ст. |331  |
|св. |313  |
|нет.|311  |
|см. |302  |
+----+-----+
only showing top 20 rows



In [ ]:
# 6. Определить устойчивые сокращения вида т.п., н.э.
complex_abbreviations_df = words_df.filter(col("word").rlike("^[а-я]{1,3}\.[а-я]{1,3}\.$"))
complex_abbreviations_df.groupBy("word") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(truncate=False)

+-------+-----+
|word   |count|
+-------+-----+
|т.е.   |60   |
|н.э.   |44   |
|т.д.   |38   |
|л.с.   |24   |
|т.к.   |24   |
|кв.м.  |23   |
|т.п.   |17   |
|т.ч.   |17   |
|с.ш.   |17   |
|т.н.   |17   |
|ю.ш.   |15   |
|р.п.   |10   |
|кв.км. |7    |
|г.г.   |7    |
|рт.ст. |6    |
|ед.ч.  |6    |
|куб.см.|6    |
|в.д.   |5    |
|и.о.   |5    |
|ст.ст. |5    |
+-------+-----+
only showing top 20 rows



In [ ]:
#Загружаем мужские и женские имена на русском языке (взял отсюда: https://github.com/Raven-SL/ru-pnames-list)
with open("female_names_rus.txt") as f:
  female_names = [name.replace("\n", "") for name in f.readlines()]

with open("male_names_rus.txt") as f:
  male_names = [name.replace("\n", "") for name in f.readlines()]

In [ ]:
# 7. Найти имена, употребляющиеся в статьях
names_df = words_df.filter(col("word").isin(female_names) | (col("word").isin(male_names))) \
    .groupBy("word") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(truncate=False)

+----------+-----+
|word      |count|
+----------+-----+
|          |12595|
|Александр |1188 |
|Александра|847  |
|Владимир  |817  |
|Петра     |760  |
|Николай   |679  |
|Пётр      |661  |
|Сергей    |601  |
|Михаил    |583  |
|Джон      |544  |
|Иван      |503  |
|Владимира |454  |
|Юрий      |407  |
|Виктор    |378  |
|Борис     |376  |
|Алексей   |369  |
|Фридрих   |334  |
|Андрей    |320  |
|Дмитрий   |319  |
|Ленина    |306  |
+----------+-----+
only showing top 20 rows



In [ ]:
# Завершение работы SparkSession
spark.stop()